# CSR Benchmark - Comparing to Bmad

In [ ]:
import numpy as np
import impact.z as IZ
from impact.z.interfaces.bmad import export_particles as tao_export_particles
from pytao import Tao

from pmd_beamphysics import ParticleGroup

from impact.tests.z.conftest import bmad_files

In [ ]:
!cat $bmad_files/csr_bench.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "csr_bench.bmad", plot="mpl")

In [ ]:
n_particle = 1000
a_norm_emit = 1.0e-6
b_norm_emit = 1.0e-6
bunch_charge = 1e-9
sig_pz = 1e-12
sig_z = 0.3e-3  # ~1ps

ds_track_step = 0.01
n_bin = 40

cmds = [
    f"set beam_init n_particle = {n_particle}",
    "set beam_init random_engine = quasi",
    "set beam_init saved_at = MARKER::*, BEGINNING, END",
    f"set beam_init a_norm_emit = {a_norm_emit}",
    f"set beam_init b_norm_emit = {b_norm_emit}",
    f"set beam_init bunch_charge = {bunch_charge}",
    f"set beam_init sig_pz = {sig_pz}",
    f"set beam_init sig_z = {sig_z}",
    f"set space_charge_com ds_track_step = {ds_track_step}",
    f"set space_charge_com n_bin = {n_bin}",
]

for cmd in cmds:
    res = tao.cmd(cmd)
    if res:
        print("Tao>", cmd)
        print(res)

In [ ]:
%time

tao.cmd("set global track_type = beam; set global track_type = single")

In [ ]:
P0 = ParticleGroup(data=tao.bunch_data("beginning"))
len(P0)
# P0.plot('t', 'energy', bins=100)

In [ ]:
P = ParticleGroup(data=tao.bunch_data("end"))
P.plot("t", "energy")

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
input = IZ.ImpactZInput.from_tao(tao, verbose=False)

In [ ]:
I = IZ.ImpactZ(
    input, use_temp_dir=False, workdir="./tmp-csr-bench", initial_particles=P0
)

In [ ]:
output = I.run(verbose=True)

In [ ]:
I.input

In [ ]:
I.output

In [ ]:
Ptao = tao_export_particles(tao, "END")

In [ ]:
import matplotlib.pyplot as plt

mc2 = I.input.reference_particle_mass
stats = I.output.stats
z = stats.z
x = stats.mean_x
y = stats.mean_y
energy = stats.mean_energy

x_tao = tao.bunch_comb("x")
y_tao = tao.bunch_comb("y")
s_tao = tao.bunch_comb("s")
p_tao = (1 + tao.bunch_comb("pz")) * tao.bunch_comb("p0c")
energy_tao = np.hypot(p_tao, mc2)

fig, axes = plt.subplots(3, figsize=(8, 6))
ax = axes[0]
ax.plot(z, x, label="Impact-Z")
ax.plot(s_tao, x_tao, "--", label="Tao")
ax.set_ylabel(r"$x$ (m)")


ax = axes[1]
ax.plot(z, y, label="Impact-Z")
ax.plot(s_tao, y_tao, "--", label="Tao")
ax.set_ylabel(r"$y$ (m)")

ax = axes[2]
ax.plot(z, energy / 1e6, label="Impact-Z")
ax.plot(s_tao, energy_tao / 1e6, "--", label="Tao")
ax.set_ylabel(r"$E$ (MeV)")

ax.set_xlabel(r"$s$ (m)")

plt.legend();